# Fulmine LABS Eyball
## Overview
This Python code implements a class wrapper around an Anomaly Detection model which can be used to visually check if an image is anomalous or not. The supported architecture for this model is 'Siamese Network'.
In order to perform reduce false negatives the code compares the image against a jury of randomly selected known good images of configurable size 'jury_size'.
If the number of jurors who vote that the image is simlar to the chosen known good image is below a configurable 'threshold' then the code returns a verdict of 'Anomalous', otherwise it returns a verdict of 'Normal'.
If an image path is not specified but screen coordinates are, these will be used instead, enabling direct integration with automated visual checking scripts.

One goal is to use this class as part of automating visual checking of a medical image (PACS) production pipeline, although it could theoretically visually check any type of image on which the model has been trained.

It also has the capability of describing the images, using GPT-4 Turbo Vision, if an OpenAI key is supplied in the 'Eyball-OpenAI_key.txt' file.

## Initialize the Eyball class

predictor = ModelPredictor(siamese_model_path, known_good_images_folder, Eyball_key, threshold, jury_size)

## Example calls

role = "You are a radiology PACS test engineer, analyzing PACS or test process related image anomalies"

image_description_directive = "If the image is obviously not a medical image, state *** ANOMALOUS ***. If it is a typical medical image as acquired by an imaging modality with no additions or enhancements, state *** NORMAL ***. Otherwise, if it is a medical image but it also clearly has textual overlays or annotations or digital or image processing artifacts that could have been added by the PACS image viewer technology, describe those features and append *** ANOMALOUS ***."

verdict = predictor.predict_siamese(test_image_path)

actual_description = predictor.describe_image(test_image_path, None, role, image_description_directive)

## Author
Duncan Henderson
Fulmine Labs LLC


In [1]:
import numpy as np
import os
import io
import cv2
from PIL import Image, ImageGrab
import logging
import random
import base64
import requests
from openai import OpenAI

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report



In [2]:
known_good_images_folder = r"D:\training_images\test\valid"
siamese_model_path = r'models\lung_ct_siamese_network_weights_043024.h5'

api_key_file='Eyball-OpenAI_key.txt'

jury_size=12
threshold = 0.5

# LLM prompts
role = "You are a radiology PACS test engineer, analyzing PACS or test process related image anomalies"
image_description_directive = "If the image is obviously not a medical image, state *** ANOMALOUS ***. If it is a typical medical image as acquired by an imaging modality with no additions or enhancements, state *** NORMAL ***. Otherwise, if it is a medical image but it also clearly has textual overlays or annotations or digital or image processing artifacts that could have been added by the PACS image viewer technology, describe those features and append *** ANOMALOUS ***."


In [3]:
import os
import numpy as np
import cv2
from PIL import Image, ImageGrab
import base64
import io
import requests
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda
import tensorflow as tf
from openai import OpenAI

class ModelPredictor:
    def __init__(self, siamese_model_path, known_good_images_folder, api_key_file='Eyball-OpenAI_key.txt', threshold=0.5, jury_size=12):
        self.siamese_model_path = siamese_model_path
        self.known_good_images_folder = known_good_images_folder
        self.api_key = self.load_api_key(api_key_file)
        self.client = OpenAI(api_key=self.api_key)
        self.siamese_model = self.load_siamese_model()
        self.threshold = threshold
        self.jury_size = jury_size
        self.known_good_images = self.preload_known_good_images()
        self.headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {self.api_key}"
        }

    def load_api_key(self, filename):
        try:
            with open(filename, 'r') as file:
                return file.read().strip()
        except FileNotFoundError:
            raise Exception(f"API key file not found: {filename}")

    def preload_known_good_images(self):
        print("Preloading known good images...")
        image_paths = []
        for root, dirs, files in os.walk(self.known_good_images_folder):
            for file in files:
                if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    full_path = os.path.join(root, file)
                    image_paths.append(full_path)
        return image_paths

    def load_siamese_model(self):
        input_shape = (152, 152, 1)
        input_a = Input(shape=input_shape)
        input_b = Input(shape=input_shape)
        base_network = self.initialize_base_network(input_shape)
        processed_a = base_network(input_a)
        processed_b = base_network(input_b)
        distance = Lambda(lambda tensors: tf.sqrt(tf.reduce_sum(tf.square(tensors[0] - tensors[1]), axis=1)))([processed_a, processed_b])
        model = Model(inputs=[input_a, input_b], outputs=distance)
        model.load_weights(self.siamese_model_path)
        print("Siamese model loaded successfully.")
        return model

    def initialize_base_network(self, input_shape):
        input = Input(shape=input_shape)
        x = Conv2D(64, (3, 3), activation='relu')(input)
        x = MaxPooling2D((2, 2))(x)
        x = Conv2D(128, (3, 3), activation='relu')(x)
        x = MaxPooling2D((2, 2))(x)
        x = Flatten()(x)
        x = Dense(128, activation='relu')(x)
        return Model(inputs=input, outputs=x)

    
    def preprocess_image(self, image=None, image_path=None):
        if image_path:
            # Load image from file path
            try:
                image = Image.open(image_path)
            except FileNotFoundError as e:
                print(f"Failed to open image at {image_path}: {e}")
                return None
            except Exception as e:
                print(f"Error processing image at {image_path}: {e}")
                return None
        if isinstance(image, np.ndarray):
            # Convert to PIL Image for consistent processing
            image = Image.fromarray(image.astype('uint8'))
    
        if image is None:
            print("No image provided for preprocessing")
            return None
    
        # Convert to grayscale and resize
        image = image.convert('L')  # Convert to grayscale
        target_size = (152, 152)
        image = image.resize(target_size)
        image = np.array(image, dtype=np.float32) / 255.0  # Normalize to [0, 1]
    
        if image.ndim == 2:  # Ensure image has 3 dimensions if it's still 2D
            image = np.expand_dims(image, -1)
    
        return image





    def capture_screen(self, coordinates):
        screenshot = ImageGrab.grab(bbox=coordinates)
        return np.array(screenshot, dtype=np.uint8)  # Ensure dtype is uint8

    def predict_siamese(self, image_path=None, coordinates=None):
        if coordinates:
            print("Capturing screen...")
            captured_image = self.capture_screen(coordinates)
            if captured_image is None:
                print("Failed to capture screen")
                return None
            image = self.preprocess_image(image=captured_image)
        elif image_path:
            image = self.preprocess_image(image_path=image_path)
            if image is None:
                print("Failed to preprocess image from path")
                return None
        else:
            raise ValueError("Either image_path or coordinates must be provided.")

    def predict_siamese(self, image_path=None, coordinates=None):
        if coordinates:
            print("Capturing screen...")
            captured_image = self.capture_screen(coordinates)
            if captured_image is None:
                print("Failed to capture screen")
                return None
            image = self.preprocess_image(image=captured_image)
        elif image_path:
            image = self.preprocess_image(image_path=image_path)
        else:
            raise ValueError("Either image_path or coordinates must be provided.")
    
        if image is None:
            print("No image to process")
            return None
    
        image = np.expand_dims(image, axis=0)  # Adjust as necessary for the model input
    
        print("Image loaded and processed, predicting...")

        votes = []
        for known_good_image_path in random.sample(self.known_good_images, min(self.jury_size, len(self.known_good_images))):
            known_good_image = self.preprocess_image(image_path=known_good_image_path)
            if known_good_image is None:
                continue  # Skip if image can't be processed
            known_good_image = np.expand_dims(known_good_image, axis=0)

            # Prepare the pair
            image_pair = [image, known_good_image]

            # Make prediction
            prediction_distance = self.siamese_model.predict(image_pair)
            is_similar = prediction_distance < self.threshold  # Threshold to determine similarity
            print(f"Comparing {image_path if image_path else 'screen capture'} with {known_good_image_path}: Distance = {prediction_distance}, Similar = {is_similar}")
            votes.append(is_similar)

        # Calculate the majority vote
        num_similar = sum(votes)
        majority_similar = num_similar > len(votes) / 2
        print(f"Total votes for 'Similar': {num_similar}/{len(votes)}. Final verdict: {'Normal' if majority_similar else 'Anomalous'}")

        return 'Normal' if majority_similar else 'Anomalous'

    def describe_image(self, image_path=None, coordinates=None, role_description="User", image_description_directive="Describe the image"):
        try:
            if coordinates:
                print("Capturing screen for description...")
                image = self.capture_screen(coordinates)
            elif image_path:
                print("Loading image from path for description...")
                image = self.preprocess_image(image_path=image_path)
            else:
                raise ValueError("Either image_path or coordinates must be provided.")
    
            if image is None:
                raise ValueError("Failed to load or process image.")
    
            print("Encoding image for API request...")
            base64_image = self.encode_image(image)
            description = self.send_image_to_api(base64_image, role_description, image_description_directive)
            print("Description received.")
            return description
        except Exception as e:
            print(f"Error in describe_image: {str(e)}")
            return None

    def encode_image(self, image):
        """Converts a numpy array image to JPEG base64."""
        try:
            if image.ndim == 3 and image.shape[2] == 1:  # Check if it's single-channel
                image = image.squeeze(-1)  # Remove the last dimension if it's single-channel
            if isinstance(image, np.ndarray):
                # Ensure the data type is uint8
                image = (image * 255).clip(0, 255).astype(np.uint8)
                # Convert numpy array to PIL Image
                if image.ndim == 2:  # Grayscale
                    image = Image.fromarray(image, 'L')
                else:
                    image = Image.fromarray(image, 'RGB')
            buffer = io.BytesIO()
            image.save(buffer, format="JPEG")
            encoded_string = base64.b64encode(buffer.getvalue()).decode('utf-8')
            return encoded_string
        except Exception as e:
            raise ValueError(f"Error encoding image: {str(e)}")

    def send_image_to_api(self, base64_image, role_description, image_description_directive):
        print("Sending image to API...")
        payload = {
            "model": "gpt-4-turbo",
            "messages": [
                {
                    "role": "system",
                    "content": role_description
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": image_description_directive
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            }
                        }
                    ]
                }
            ],
            "max_tokens": 300
        }
        response = requests.post("https://api.openai.com/v1/chat/completions", headers=self.headers, json=payload)
        if response.status_code != 200:
            print("Error from API:", response.status_code, response.text)
            return None

        try:
            description = response.json()['choices'][0]['message']['content']
            return description
        except KeyError as e:
            print("Failed to parse API response:", response.json())
            raise e



In [4]:
predictor = ModelPredictor(siamese_model_path, known_good_images_folder, api_key_file, threshold, jury_size)



Siamese model loaded successfully.
Preloading known good images...


In [5]:
# Capture and evaluate an area of the screen ...

left = 10
right = 500
top = 10
bottom = 200

siamese_result = predictor.predict_siamese(coordinates=(left, top, right, bottom))
print ("Siamese result", siamese_result)

predictor.describe_image(coordinates=(left, top, right, bottom), role_description=role, image_description_directive=image_description_directive)

Capturing screen...
Image loaded and processed, predicting...
1/1 [==============================] - 0s 84ms/step
Comparing screen capture with D:\training_images\test\valid\randomized_wl\cropped\8f67f992-5669-4488-9ff3-0ddd75100b32_1.png: Distance = [1.0162967], Similar = [False]
1/1 [==============================] - 0s 21ms/step
Comparing screen capture with D:\training_images\test\valid\randomized_wl\cropped\132d8a36-2862-4d8b-9ee6-bc74f2ea510c_1.png: Distance = [1.1367092], Similar = [False]
1/1 [==============================] - 0s 16ms/step
Comparing screen capture with D:\training_images\test\valid\zoomed\randomized_wl\c25d266a-a314-40ca-8b2d-5685444c9374_0.png: Distance = [1.1367092], Similar = [False]
1/1 [==============================] - 0s 20ms/step
Comparing screen capture with D:\training_images\test\valid\randomized_wl\cropped\9c59a8e8-de3d-44e7-9a91-7564417c63be_0.png: Distance = [1.1367092], Similar = [False]
1/1 [==============================] - 0s 12ms/step
Compari

'*** ANOMALOUS ***'

In [6]:
# Or (from here on) pass in a captured and saved file

test_image_path = r'C:\temp\engineer_typing3.png'
print("Model predicts", predictor.predict_siamese(image_path=test_image_path))

actual_description = predictor.describe_image(image_path=test_image_path, role_description=role, image_description_directive=image_description_directive)
print ("LLM description", actual_description)


Image loaded and processed, predicting...
1/1 [==============================] - 0s 45ms/step
Comparing C:\temp\engineer_typing3.png with D:\training_images\test\valid\zoomed\randomized_wl\20449f88-1d2e-422a-bcfe-08c204d8d9d2_0.png: Distance = [0.78976893], Similar = [False]
1/1 [==============================] - 0s 56ms/step
Comparing C:\temp\engineer_typing3.png with D:\training_images\test\valid\dummy_class\9c35c6da-c3ce-40d6-9b5c-b8757939def7.png: Distance = [1.0572661], Similar = [False]
1/1 [==============================] - 0s 44ms/step
Comparing C:\temp\engineer_typing3.png with D:\training_images\test\valid\randomized_wl\cropped\67c79cad-9871-4651-8193-d0da271f9504_1.png: Distance = [1.0572661], Similar = [False]
1/1 [==============================] - 0s 49ms/step
Comparing C:\temp\engineer_typing3.png with D:\training_images\test\valid\dummy_class\03173cc0-7bb3-4125-9ee9-9a272cf17ebc.png: Distance = [1.0572661], Similar = [False]
1/1 [==============================] - 0s 50ms

In [7]:
test_image_path = r'D:\Custom_invalid\cat.jpg'
print("Model predicts", predictor.predict_siamese(test_image_path))

predictor.describe_image(test_image_path, None, role, image_description_directive)
print ("LLM description", predictor.describe_image(test_image_path, None, role, image_description_directive))


Image loaded and processed, predicting...
1/1 [==============================] - 0s 78ms/step
Comparing D:\Custom_invalid\cat.jpg with D:\training_images\test\valid\randomized_wl\cropped\5590bf43-d43f-4046-9b1e-d85893efc855_1.png: Distance = [1.168402], Similar = [False]
1/1 [==============================] - 0s 63ms/step
Comparing D:\Custom_invalid\cat.jpg with D:\training_images\test\valid\dummy_class\4189c278-43b2-4a25-8734-e2aacf6b831f.png: Distance = [1.168402], Similar = [False]
1/1 [==============================] - 0s 46ms/step
Comparing D:\Custom_invalid\cat.jpg with D:\training_images\test\valid\randomized_wl\cropped\d0d07cdb-2ebd-45d8-bdcc-4d98f4b9603d_0.png: Distance = [0.9997449], Similar = [False]
1/1 [==============================] - 0s 46ms/step
Comparing D:\Custom_invalid\cat.jpg with D:\training_images\test\valid\randomized_wl\cropped\642e1050-3997-477a-a624-0fbcb93e1209_0.png: Distance = [1.168402], Similar = [False]
1/1 [==============================] - 0s 41ms/st

In [8]:
test_image_path = r'D:\custom_test_valid\internet_27f6574b96deb965217cff1aac35fc_gallery.jpg'
print("Model predicts", predictor.predict_siamese(test_image_path))

predictor.describe_image(test_image_path, None, role, image_description_directive)
print ("LLM description", predictor.describe_image(test_image_path, None, role, image_description_directive))


Image loaded and processed, predicting...
1/1 [==============================] - 0s 52ms/step
Comparing D:\custom_test_valid\internet_27f6574b96deb965217cff1aac35fc_gallery.jpg with D:\training_images\test\valid\dummy_class\8b932300-bb73-48c1-8656-34b993ad99f7.png: Distance = [0.], Similar = [ True]
1/1 [==============================] - 0s 44ms/step
Comparing D:\custom_test_valid\internet_27f6574b96deb965217cff1aac35fc_gallery.jpg with D:\training_images\test\valid\dummy_class\41026ec9-e191-4568-aeb6-f51c44bf0915.png: Distance = [0.], Similar = [ True]
1/1 [==============================] - 0s 42ms/step
Comparing D:\custom_test_valid\internet_27f6574b96deb965217cff1aac35fc_gallery.jpg with D:\training_images\test\valid\randomized_wl\cropped\d0a983dd-f6af-4875-8b64-cb31a4ab5303_1.png: Distance = [0.871497], Similar = [False]
1/1 [==============================] - 0s 47ms/step
Comparing D:\custom_test_valid\internet_27f6574b96deb965217cff1aac35fc_gallery.jpg with D:\training_images\test

In [9]:
test_image_path = r'D:\custom_test_valid\istockphoto-493741910-612x612.jpg'
print("Model predicts", predictor.predict_siamese(test_image_path))

predictor.describe_image(test_image_path, None, role, image_description_directive)
print ("LLM description", predictor.describe_image(test_image_path, None, role, image_description_directive))


Image loaded and processed, predicting...
1/1 [==============================] - 0s 42ms/step
Comparing D:\custom_test_valid\istockphoto-493741910-612x612.jpg with D:\training_images\test\valid\dummy_class\52a6abd1-5252-47a1-bc4e-e43ee9e2bc98.png: Distance = [0.12309527], Similar = [ True]
1/1 [==============================] - 0s 45ms/step
Comparing D:\custom_test_valid\istockphoto-493741910-612x612.jpg with D:\training_images\test\valid\zoomed\randomized_wl\9fec6bc0-c9f3-4b1d-88ea-c86d9aa3eaae_0.png: Distance = [0.35541472], Similar = [ True]
1/1 [==============================] - 0s 43ms/step
Comparing D:\custom_test_valid\istockphoto-493741910-612x612.jpg with D:\training_images\test\valid\randomized_wl\cropped\bd3aceec-b75c-4061-9fde-e276eb58b5d7_0.png: Distance = [0.14590386], Similar = [ True]
1/1 [==============================] - 0s 42ms/step
Comparing D:\custom_test_valid\istockphoto-493741910-612x612.jpg with D:\training_images\test\valid\zoomed\randomized_wl\2c3621c1-cfc7-4

In [10]:
test_image_path = r'D:\custom_test_valid\low-dose-lung-cancer-screening-with-lung-nodules.jpg'
print("Model predicts", predictor.predict_siamese(test_image_path))

predictor.describe_image(test_image_path, None, role, image_description_directive)
print ("LLM description", predictor.describe_image(test_image_path, None, role, image_description_directive))


Image loaded and processed, predicting...
1/1 [==============================] - 0s 42ms/step
Comparing D:\custom_test_valid\low-dose-lung-cancer-screening-with-lung-nodules.jpg with D:\training_images\test\valid\dummy_class\a619f51c-ea42-446a-96ac-8dfde1932677.png: Distance = [0.71923923], Similar = [False]
1/1 [==============================] - 0s 43ms/step
Comparing D:\custom_test_valid\low-dose-lung-cancer-screening-with-lung-nodules.jpg with D:\training_images\test\valid\zoomed\randomized_wl\9b15c6da-e406-439a-9904-b47d81915396_0.png: Distance = [0.71923923], Similar = [False]
1/1 [==============================] - 0s 41ms/step
Comparing D:\custom_test_valid\low-dose-lung-cancer-screening-with-lung-nodules.jpg with D:\training_images\test\valid\zoomed\randomized_wl\897d99e5-442b-418c-bea3-61e457eb773c_1.png: Distance = [0.71923923], Similar = [False]
1/1 [==============================] - 0s 42ms/step
Comparing D:\custom_test_valid\low-dose-lung-cancer-screening-with-lung-nodules.

In [11]:
test_image_path = r'D:\custom_invalid\istockphoto-with_arrow.jpg'
print("Model predicts", predictor.predict_siamese(test_image_path))

predictor.describe_image(test_image_path, None, role, image_description_directive)
print ("LLM description", predictor.describe_image(test_image_path, None, role, image_description_directive))


Image loaded and processed, predicting...
1/1 [==============================] - 0s 47ms/step
Comparing D:\custom_invalid\istockphoto-with_arrow.jpg with D:\training_images\test\valid\dummy_class\f29b6cc8-4da3-4884-af42-2fe29605bca5.png: Distance = [0.5946167], Similar = [False]
1/1 [==============================] - 0s 42ms/step
Comparing D:\custom_invalid\istockphoto-with_arrow.jpg with D:\training_images\test\valid\randomized_wl\cropped\875e82e7-9249-4229-9a0f-f4e4500d2d46_0.png: Distance = [0.42301986], Similar = [ True]
1/1 [==============================] - 0s 43ms/step
Comparing D:\custom_invalid\istockphoto-with_arrow.jpg with D:\training_images\test\valid\zoomed\randomized_wl\4db13d59-613f-4f98-b708-451930704fae_0.png: Distance = [0.8484544], Similar = [False]
1/1 [==============================] - 0s 41ms/step
Comparing D:\custom_invalid\istockphoto-with_arrow.jpg with D:\training_images\test\valid\zoomed\randomized_wl\e9842c6e-2e6a-4868-94de-dc9a22044e40_1.png: Distance = [0

In [12]:
test_image_path = r'D:\custom_invalid\Lung_abscess_-_CT_with_overlay.jpg'
print("Model predicts", predictor.predict_siamese(test_image_path))

predictor.describe_image(test_image_path, None, role, image_description_directive)
print ("LLM description", predictor.describe_image(test_image_path, None, role, image_description_directive))


Image loaded and processed, predicting...
1/1 [==============================] - 0s 41ms/step
Comparing D:\custom_invalid\Lung_abscess_-_CT_with_overlay.jpg with D:\training_images\test\valid\dummy_class\6f3942bd-df11-4f4b-91be-9ec418be7ff2.png: Distance = [1.1311321], Similar = [False]
1/1 [==============================] - 0s 43ms/step
Comparing D:\custom_invalid\Lung_abscess_-_CT_with_overlay.jpg with D:\training_images\test\valid\zoomed\randomized_wl\1a4fa34b-1f56-4a24-a44a-bf06948e89bb_0.png: Distance = [1.1311321], Similar = [False]
1/1 [==============================] - 0s 41ms/step
Comparing D:\custom_invalid\Lung_abscess_-_CT_with_overlay.jpg with D:\training_images\test\valid\dummy_class\27134961-5a06-4abb-8ddb-65d008056401.png: Distance = [1.1311321], Similar = [False]
1/1 [==============================] - 0s 42ms/step
Comparing D:\custom_invalid\Lung_abscess_-_CT_with_overlay.jpg with D:\training_images\test\valid\randomized_wl\cropped\0ea7fcc5-b9a1-428d-9904-ab8106c54bc4_

In [13]:
test_image_path = r'D:\Custom_invalid\augmented_0abe42cc-623a-46f2-91ee-be4f339ff73b.png'
print("Model predicts", predictor.predict_siamese(test_image_path))

predictor.describe_image(test_image_path, None, role, image_description_directive)
print ("LLM description", predictor.describe_image(test_image_path, None, role, image_description_directive))


Image loaded and processed, predicting...
1/1 [==============================] - 0s 41ms/step
Comparing D:\Custom_invalid\augmented_0abe42cc-623a-46f2-91ee-be4f339ff73b.png with D:\training_images\test\valid\randomized_wl\cropped\d566f961-422d-4b79-8080-4488b9739980_1.png: Distance = [0.6476503], Similar = [False]
1/1 [==============================] - 0s 41ms/step
Comparing D:\Custom_invalid\augmented_0abe42cc-623a-46f2-91ee-be4f339ff73b.png with D:\training_images\test\valid\randomized_wl\cropped\1779f7eb-e492-4f7c-b066-233c3ec30f96_1.png: Distance = [0.6476503], Similar = [False]
1/1 [==============================] - 0s 41ms/step
Comparing D:\Custom_invalid\augmented_0abe42cc-623a-46f2-91ee-be4f339ff73b.png with D:\training_images\test\valid\randomized_wl\cropped\0bac8e20-f662-4849-82c9-3a963566fca0_0.png: Distance = [0.4070547], Similar = [ True]
1/1 [==============================] - 0s 44ms/step
Comparing D:\Custom_invalid\augmented_0abe42cc-623a-46f2-91ee-be4f339ff73b.png with 

In [14]:
test_image_path = r'C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio_hairlines.png'
print("Model predicts", predictor.predict_siamese(test_image_path))

predictor.describe_image(test_image_path, None, role, image_description_directive)
print ("LLM description", predictor.describe_image(test_image_path, None, role, image_description_directive))


Image loaded and processed, predicting...
1/1 [==============================] - 0s 41ms/step
Comparing C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio_hairlines.png with D:\training_images\test\valid\zoomed\randomized_wl\695a587c-e89c-4045-8e4c-21a65507f88a_1.png: Distance = [0.98369527], Similar = [False]
1/1 [==============================] - 0s 43ms/step
Comparing C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio_hairlines.png with D:\training_images\test\valid\zoomed\randomized_wl\b6969d0c-48df-4221-8ac0-c42e9eb452fa_1.png: Distance = [0.98369527], Similar = [False]
1/1 [==============================] - 0s 43ms/step
Comparing C:\temp\medical_image_zoomed_more_resized_modified_aspect_ratio_hairlines.png with D:\training_images\test\valid\randomized_wl\cropped\cef21e5b-d290-4d4d-8b56-762ec9e7bfbe_0.png: Distance = [0.98369527], Similar = [False]
1/1 [==============================] - 0s 42ms/step
Comparing C:\temp\medical_image_zoomed_more_resized_modi

In [15]:
test_image_path = r'D:\Custom_invalid\internet-gettyimages-1320918955-612x612_small_label.jpg'
print("Model predicts", predictor.predict_siamese(test_image_path))

predictor.describe_image(test_image_path, None, role, image_description_directive)
print ("LLM description", predictor.describe_image(test_image_path, None, role, image_description_directive))

Image loaded and processed, predicting...
1/1 [==============================] - 0s 42ms/step
Comparing D:\Custom_invalid\internet-gettyimages-1320918955-612x612_small_label.jpg with D:\training_images\test\valid\randomized_wl\cropped\4a889dfd-abc5-44e4-ae12-bdb47693addf_1.png: Distance = [0.77528584], Similar = [False]
1/1 [==============================] - 0s 40ms/step
Comparing D:\Custom_invalid\internet-gettyimages-1320918955-612x612_small_label.jpg with D:\training_images\test\valid\randomized_wl\cropped\13231a60-86f8-4f4a-b695-f4b252425385_1.png: Distance = [0.4195814], Similar = [ True]
1/1 [==============================] - 0s 39ms/step
Comparing D:\Custom_invalid\internet-gettyimages-1320918955-612x612_small_label.jpg with D:\training_images\test\valid\zoomed\randomized_wl\5c2d7756-a312-42f8-a056-a5c171d95446_1.png: Distance = [0.76317704], Similar = [False]
1/1 [==============================] - 0s 41ms/step
Comparing D:\Custom_invalid\internet-gettyimages-1320918955-612x612_

In [16]:
test_image_path = r'D:\Custom_test_valid\internet-gettyimages-1322138871-612x612.jpg'
print("Model predicts", predictor.predict_siamese(test_image_path))

predictor.describe_image(test_image_path, None, role, image_description_directive)
print ("LLM description", predictor.describe_image(test_image_path, None, role, image_description_directive))


Image loaded and processed, predicting...
1/1 [==============================] - 0s 35ms/step
Comparing D:\Custom_test_valid\internet-gettyimages-1322138871-612x612.jpg with D:\training_images\test\valid\randomized_wl\cropped\4492da1d-77fa-44d5-8376-cdccaf48e839_1.png: Distance = [0.09772173], Similar = [ True]
1/1 [==============================] - 0s 52ms/step
Comparing D:\Custom_test_valid\internet-gettyimages-1322138871-612x612.jpg with D:\training_images\test\valid\dummy_class\c6291547-c197-4ab0-9099-4694445d5ebf.png: Distance = [0.45649433], Similar = [ True]
1/1 [==============================] - 0s 41ms/step
Comparing D:\Custom_test_valid\internet-gettyimages-1322138871-612x612.jpg with D:\training_images\test\valid\zoomed\randomized_wl\f4ce427d-3bea-43ae-b8ce-408b9ebe9080_1.png: Distance = [0.45649433], Similar = [ True]
1/1 [==============================] - 0s 44ms/step
Comparing D:\Custom_test_valid\internet-gettyimages-1322138871-612x612.jpg with D:\training_images\test\va

In [17]:
def evaluate_methods_simplified(base_folder, sample_size=40, jury_size=12, role="User", image_description_directive="Describe the image"):
    valid_folder = os.path.join(base_folder, 'valid')
    invalid_folder = os.path.join(base_folder, 'invalid')

    # Ensure directories exist
    if not os.path.exists(valid_folder) or not os.path.exists(invalid_folder):
        raise ValueError("One or more image directories do not exist.")

    valid_images = random.sample(os.listdir(valid_folder), min(sample_size, len(os.listdir(valid_folder))))
    invalid_images = random.sample(os.listdir(invalid_folder), min(sample_size, len(os.listdir(invalid_folder))))
    
    predictions_siamese = []
    predictions_gpt = []
    actuals = [1] * len(valid_images) + [0] * len(invalid_images)

    for filename in valid_images + invalid_images:
        folder = valid_folder if filename in valid_images else invalid_folder
        image_path = os.path.join(folder, filename)

        siamese_result = predictor.predict_siamese(image_path)
        print ("Siamese prediction", siamese_result)
        predictions_siamese.append(1 if siamese_result == 'Normal' else 0)

        description_result = predictor.describe_image(image_path, None, role, image_description_directive)
        print ("LLM prediction", description_result)
        predictions_gpt.append(1 if 'NORMAL' in description_result.upper() else 0)

    accuracy_s = accuracy_score(actuals, predictions_siamese)
    precision_s = precision_score(actuals, predictions_siamese)
    recall_s = recall_score(actuals, predictions_siamese)
    f1_s = f1_score(actuals, predictions_siamese)
    
    accuracy_g = accuracy_score(actuals, predictions_gpt)
    precision_g = precision_score(actuals, predictions_gpt)
    recall_g = recall_score(actuals, predictions_gpt)
    f1_g = f1_score(actuals, predictions_gpt)

    print('Evaluation Results - Siamese Model:', {'accuracy': accuracy_s, 'precision': precision_s, 'recall': recall_s, 'f1': f1_s})
    print('Evaluation Results - GPT Model:', {'accuracy': accuracy_g, 'precision': precision_g, 'recall': recall_g, 'f1': f1_g})

    return {
        "siamese": {"accuracy": accuracy_s, "precision": precision_s, "recall": recall_s, "f1": f1_s},
        "gpt": {"accuracy": accuracy_g, "precision": precision_g, "recall": recall_g, "f1": f1_g}
    }



In [18]:
# Example call
base_folder = r'D:\model_comparison_test'
sample_size = 40
try:
    results = evaluate_methods_simplified(base_folder, sample_size, jury_size, role, image_description_directive)
    print("Evaluation Results:", results)
except Exception as e:
    print("Error during evaluation:", str(e))


Image loaded and processed, predicting...
1/1 [==============================] - 0s 35ms/step
Comparing D:\model_comparison_test\valid\fab8297a-ff45-496b-8608-c37a7636042e.png with D:\training_images\test\valid\dummy_class\becc8af6-f056-490a-94af-5e77f0d77011.png: Distance = [0.], Similar = [ True]
1/1 [==============================] - 0s 50ms/step
Comparing D:\model_comparison_test\valid\fab8297a-ff45-496b-8608-c37a7636042e.png with D:\training_images\test\valid\dummy_class\0dca7b33-de4c-4389-981b-0b9b3717fe88.png: Distance = [0.], Similar = [ True]
1/1 [==============================] - 0s 40ms/step
Comparing D:\model_comparison_test\valid\fab8297a-ff45-496b-8608-c37a7636042e.png with D:\training_images\test\valid\randomized_wl\cropped\becc8af6-f056-490a-94af-5e77f0d77011_1.png: Distance = [0.04846442], Similar = [ True]
1/1 [==============================] - 0s 41ms/step
Comparing D:\model_comparison_test\valid\fab8297a-ff45-496b-8608-c37a7636042e.png with D:\training_images\test\v